In [2]:
import numpy as np
import random
import pandas as pd

def load_iris(training_ratio):
    """Load the iris data.

    Args:
        training_ratio: the ratio of examples that go into the training set
    Returns:
        a tuple of numpy matrices, the first in the tuple is the training 
        data, second is test data. Each matrix row represents a data point 
        as a row vector: the first element of the row vector corresponds to 
        the label and the following elements correspond to attributes.
    """
    random.seed(1) # get same data every time
    iris_labels = {'Iris-setosa': 0, 'Iris-versicolor' : 1, 'Iris-virginica' : 2}
    f = open('data/iris.data', 'r')

    training = None
    test = None
    lines = f.readlines()
    train_index = int(len(lines)*training_ratio)
    random.shuffle(lines)
    for k, line in enumerate(lines):
        data = line.split(',')
        vector = [int(iris_labels[data[-1].rstrip('\n')])]+ [float(i) for i in data[0:-1]]
        vector = np.array(vector)
        if k < train_index:
            if training is None:
                training = vector
            else:
                training = np.vstack((training, vector))
        else:
            if test is None:
                test = vector
            else:
                test = np.vstack((test, vector))
    return (training, test)

class TreeNode(object):
    def __init__(self, ids = None, children = [], depth = 0):
        self.ids = ids           # index of data in this node
        self.depth = depth       # distance to root node
        self.split_attribute = None # which attribute is chosen, it non-leaf
        self.children = children # list of its child nodes
        self.order = None       # order of values of split_attribute in children
        self.label = None       # label of node if it is a leaf
    
    #this function will set the split_attribute and the values of 
    #split_attribute in children corresponding to the node
    def set_properties(self, split_attribute, order):
        self.split_attribute = split_attribute
        self.order = order
    
    #this function will set the label for the leaf node
    def set_label(self, label):
        self.label = label


def entropy(freq, model):
    #this function will calculate the entropy values for each values in a particular feature
    #if model = "gini", it will returns value based on gini index formula
    #if model = "entropy", it will returns value based on entropy formula
    prob_0 = freq/float(freq.sum())
    if model == "entropy":
    #calulate using entropy
        return -np.sum(prob_0*np.log(prob_0))
    if model == "gini":
    #calulate using gini index
        return 2*np.prod(prob_0)        
        
class DecisionTree(object):
    def __init__(self, model = "entropy"):
        self.root = None 
        self.max_depth = 0
        self.Ntrain = 0
        self.model = model
    def fit(self, data, target):
        self.Ntrain = data.count()[1]
        self.data = data #this is a matrix of N data points with each column is a feature
        self.attributes = list(data.columns) #list of name of the features
        self.target = target #this is a matrix of N data points, each is a label
        self.labels = target.unique() #list of unique labels
        self.max_depth = len(self.attributes) #the max_depth will be set to be the number of features
        
        ids = range(self.Ntrain)
        self.root = TreeNode(ids = ids, depth = 0)
        queue = [self.root]
        while queue:
            node = queue.pop()
            if node.depth < self.max_depth:
                #function _split will both store best_attribute after entropy calculation in the node
                #and returns a list of children, with each children is a split accordingly
                node.children = self._split(node)
                if not node.children: #leaf node, then set label and store into the node
                    self._set_label(node)
                queue += node.children #add the children to the node to continue growing the tree
            else:
                self._set_label(node) #if the depth is at its max, then set label corresponding to the node
        
    def _entropy(self, ids):
        # calculate entropy of a node with index ids
        if len(ids) == 0: return 0
        freq = np.array(self.target[ids].value_counts())
        return entropy(freq, self.model)
    
    def _set_label(self, node):
        # find label for a node if it is a leaf
        # simply chose by major voting 
        target_ids = [i for i in node.ids]  # target is a series variable
        node.set_label(self.target[target_ids].mode()[0]) # most frequent label
        
    def _split(self, node):
        ids = node.ids 
        best_splits = []
        best_attribute = None
        order = None
        sub_data = self.data.iloc[ids, :] #split data according to the list of ids stored in the node
        
        minG = 1
        for i, att in enumerate(self.attributes):
            values = self.data.iloc[ids, i].unique().tolist() 
            if len(values) == 1: continue # homogenous for this attribute feature, then continue
            splits = []
            for val in values: 
                #for each value in the attribute, we will obtain the index of all data that have the same value 
                #and add them into a list
                sub_ids = sub_data.index[sub_data[att] == val].tolist() 
                splits.append([sub_id for sub_id in sub_ids])
            if min(map(len, splits)) == 0: continue #if the split data is empty, then continue
            Entropy = 0
            for split in splits:
                Entropy += len(split)*self._entropy(split)/len(ids)
            #we will choose the attribute with the minimum entropy value
            if Entropy < minG:
                minG = Entropy
                best_attribute = att
                best_splits = splits
                order = values
        #set the best_attribute for the node
        node.set_properties(best_attribute, order)
        #create a list of children nodes according to split ids obtained from best_splits
        child_nodes = [TreeNode(ids = split,
                     depth = node.depth + 1) for split in best_splits]
        return child_nodes
    
    def predict(self, new_data):
        
        #param new_data: a new dataframe, each row is a datapoint
        #return: predicted labels for each row
        npoints = new_data.count()[1]
        labels = [None]*npoints
        for n in range(npoints):
            x = new_data.iloc[n, :] # one point 
            # start from root and recursively travel if not meet a leaf 
            node = self.root
            while node.children: 
                #if the value in test set is not covered in training set, default to pick the first children node
                if x[node.split_attribute] not in node.order:
                    node = node.children[0]
                else:
                    node = node.children[node.order.index(x[node.split_attribute])]                
            labels[n] = node.label
            
        return labels

def get_score(predict, target):
    #this function returns the accuracy of our decision tree
    count = 0
    for i in range(target.shape[0]):
        if target[i] == predict[i]:
            count+=1
    return count/target.shape[0]
    
    
def main():
    training, test = load_iris(0.6)
    df1 = pd.DataFrame(training)
    df2 = pd.DataFrame(test)
    X_train = df1.iloc[:, 1:]
    y_train = df1.iloc[:, 0]
    X_test = df2.iloc[:, 1:]
    y_test = df2.iloc[:, 0]
    tree1 = DecisionTree(model = "entropy")
    tree1.fit(X_train, y_train)
    prediction1 = tree1.predict(X_test)
    tree2  = DecisionTree(model= "gini")
    tree2.fit(X_train, y_train)
    prediction2 = tree2.predict(X_test)
    return (get_score(prediction1, y_test),get_score(prediction2, y_test))


entropy_score, gini_score = main()
print("The tree decision score with entropy is ",entropy_score)
print("The tree decision score with gini index is ",gini_score)

The tree decision score with entropy is  0.8
The tree decision score with gini index is  0.6
